In [ ]:
### DISCLAIMER: 
### FOR ALL TAX-RELATED QUESTIONS AND CONCERNS PLEASE ASK A CERTIFIED ACCOUNTANT

# This notebook analyzes security related transactions for a property
# All security related transactions should have the label/category "Security"

# In mint a Security received from a tenant is set to a credit transaction 
#(even though this is a liability carried by the property owner)

# Security returned to a tenant is set to a debit transaction

# When some security is kept, two transactions should be manually entered:
# The first transaction should be debit labeled "Security" to indicate that
# the liability to return the security to the tenant has been removed
# Additional transaction(s) should be a credit and categorized for what the 
# kept security was used for so that the income/expense is properly calculate

# Example: A tenant paid a $1000 deposit.  When she moved out $700 was returned
# to her, $200 was kept to offset Utilities overages, and $100 was used for cleaning
# This would require the following in the transaction data
# -- "Security" credit for $1000 to represent the initial tenant payment
# -- "Security" debit of $700 to represent the amount returned to the tenant
# -- "Security" debit of $300 to represent the "Kept Security" for utiltiies and cleaning
# -- "Utilties" credit of $200 to represent the utilities expense offset from the kept security
# -- "Maintenance" credit of $100 to represent the maintenance expense offset 

# Note that this notebook does not work with categories other than "Security", but the overall
# Income/Expense analysis requires the additional expense offsets

import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

# Prior to running move your MINT.com transactions.csv into this project directory
PATH_TO_YOUR_TRANSACTIONS = "transactions.csv"
PATH_TO_RENTS = "rents.csv"

# Set the name of the column where the transactions are categorized
# In Mint this is usually "Category".  I use Mint's category to link 
# transactions to a property and the add a seperate "Label" column
# after exporting the data for a property from Min
CATEGORY = "Label"


# Load the transaction data from the csv into a dataframe
parse_dates = ['Date']
df = pd.read_csv(PATH_TO_YOUR_TRANSACTIONS, parse_dates=parse_dates)
df.set_index(['Date'], inplace=True)
df['Amount'] = df['Amount'].astype(float)

# Get rid of Mint columns that we don't care about
df.drop(['Original Description', 'Notes', 'Category', 'Account Name'], axis=1, inplace=True)
# Ignore everything other than the security related transactions
df = df[(df[CATEGORY] == 'Security') | (df[CATEGORY] == 'Security-Income')]
df.sort_index(inplace=True)
# Show all the security related transations
df

In [ ]:
## Show the Current Security Liability for Each Unit

# Read in the rents data so we know the units
rents = pd.read_csv(PATH_TO_RENTS)
rents.set_index(['Year'], inplace=True)

for unit in rents.columns:
    unit_df = df[df['Unit'] == unit]
    collected = unit_df[(unit_df['Transaction Type'] == 'credit')]['Amount'].sum()
    returned = unit_df[(unit_df['Transaction Type'] == 'debit')]['Amount'].sum()
    print('Current Liability on Unit '+str(unit)+': ${:,.2f}'.format(collected-returned))
